<a href="https://colab.research.google.com/github/ewakobrzynska/MachineLearning/blob/main/065_SVM_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [1]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = (kernel + 1) ** d
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [2]:
def choose_set_for_label(data_set, labels, label):
    train_data_set = data_set
    train_labels = np.where(labels == label, 1, -1)
    return train_data_set, train_labels


In [3]:
def get_labels_count(labels):
    return len(np.unique(labels))


Use the code that we have implemented earlier:

In [7]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)
    objects_count = len(train_data_set)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import cvxopt
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt
from itertools import combinations

iris = load_iris()
data_set = iris.data
labels = iris.target
# Assuming train_data_set and train_labels are defined
train_data_set, test_data_set, train_labels, test_labels = train_test_split(data_set, labels, test_size=0.3, random_state=42)
num_classes = get_labels_count(train_labels)
classifiers = []

for label in range(num_classes):
    binary_train_data_set, binary_train_labels = choose_set_for_label(train_data_set, train_labels, label)
    lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(binary_train_data_set, binary_train_labels, kernel_type='poly')
    classifiers.append((lambdas, support_vectors, support_vectors_id, b, targets, vector_number))

def classify_multiclass(test_data_set, classifiers):
    predictions = np.zeros((test_data_set.shape[0], len(classifiers)))
    for i, (lambdas, support_vectors, support_vectors_id, b, targets, vector_number) in enumerate(classifiers):
        kernel = np.dot(test_data_set, support_vectors.T)
        kernel = (kernel + 1) ** 3
        y = np.zeros((test_data_set.shape[0], 1))
        for j in range(test_data_set.shape[0]):
            for k in range(vector_number):
                y[j] += lambdas[k] * targets[k] * kernel[j, k]
            y[j] += b
        predictions[:, i] = y.ravel()
    return np.argmax(predictions, axis=1)

predicted = classify_multiclass(test_data_set, classifiers)
print(accuracy_score(predicted, test_labels))


     pcost       dcost       gap    pres   dres
 0: -2.0117e+00 -2.3038e+03  6e+03  3e-01  1e-13
 1:  1.9188e+00 -1.7102e+02  3e+02  1e-02  9e-14
 2:  1.2781e+00 -3.5437e+00  6e+00  1e-04  9e-15
 3:  2.2764e-01 -3.7188e-01  6e-01  6e-07  3e-15
 4:  1.3590e-02 -7.2173e-02  9e-02  2e-16  2e-15
 5: -9.8857e-03 -2.0421e-02  1e-02  2e-16  6e-16
 6: -1.1871e-02 -1.5512e-02  4e-03  2e-16  3e-16
 7: -1.2996e-02 -1.6100e-02  3e-03  2e-16  3e-16
 8: -1.4904e-02 -1.5194e-02  3e-04  2e-16  4e-16
 9: -1.4988e-02 -1.5028e-02  4e-05  2e-16  4e-16
10: -1.5003e-02 -1.5005e-02  2e-06  2e-16  4e-16
11: -1.5004e-02 -1.5004e-02  1e-07  2e-16  4e-16
12: -1.5004e-02 -1.5004e-02  4e-09  2e-16  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.6391e+00 -2.3068e+03  6e+03  3e-01  9e-14
 1:  2.0460e+00 -1.3429e+02  2e+02  8e-03  9e-14
 2:  1.1171e+00 -7.6090e+00  1e+01  4e-04  7e-15
 3:  3.1672e-01 -5.2100e-01  9e-01  3e-06  8e-15
 4:  2.5144e-02 -9.1996e-02  1e-01  2e-16  2e-1